In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

qdrant_client = QdrantClient(":memory:")

##As our vector size is 4 so i am defining size as 4
qdrant_client.create_collection(
    collection_name="demo_collection1",
    vectors_config=VectorParams(size=4, distance=Distance.COSINE),
)

/home/prod/miniforge3/envs/excel-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
##Suppose we have this payload, i.e data which we want to feed in the qdrant vectordatabase
data_to_ingest = [
  { "id": 1, "city": "London", "color": "green" },
  { "id": 2, "city": "London", "color": "red" },
  { "id": 3, "city": "London", "color": "blue" },
  { "id": 4, "city": "Berlin", "color": "red" },
  { "id": 5, "city": "Moscow", "color": "green" },
  { "id": 6, "city": "Moscow", "color": "blue" }
]

In [3]:
from qdrant_client import models


qdrant_client.upsert(
    collection_name="demo_collection1",
    points=[
        models.PointStruct(
            id=1,
            vector=[0.05, 0.61, 0.76, 0.74],
            payload={
                "city": "London",
                "color": "green"
            },
        ),
        models.PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={
                "city": "London", 
                "color": "red"
            },
        ),
        models.PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={
                "city": "London", 
                "color": "blue"
            },
        ),
        models.PointStruct(
            id=4,
            vector=[0.40, 0.57, 0.37, 0.44],
            payload={
                "city": "Berlin",
                "color": "red"
            },
        ),
        models.PointStruct(
            id=5,
            vector=[0.66, 0.85, 0.67, 0.54],
            payload={
                "city": "Moscow",
                "color": "green"
            },
        ),
        models.PointStruct(
            id=6,
            vector=[0.69, 0.15, 0.27, 0.84],
            payload={
                "city": "Moscow",
                "color": "blue"
            },
        ),
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Filtering--
-- Using must: its equivalent to AND operation it will be return those records which satisfy all condition of list

In [4]:
from qdrant_client import models

qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(value="London"),
            ),
            models.FieldCondition(
                key="color",
                match=models.MatchValue(value="red"),
            ),
        ]
    ),
)

([Record(id=2, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 None)

In [5]:
from qdrant_client import models

qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(value="London"),
            ),
            models.FieldCondition(
                key="color",
                match=models.MatchValue(value="red"),
            ),
        ]
    ),
    with_vectors=True,
)

([Record(id=2, payload={'city': 'London', 'color': 'red'}, vector=[0.16881054639816284, 0.7196660041809082, 0.6663573980331421, 0.09773242473602295], shard_key=None, order_value=None)],
 None)

-- Using Should: the clause becomes true if at least one condition listed inside should is satisfied. In this sense, should is equivalent to the operator OR.

In [6]:
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(value="London"),
            ),
            models.FieldCondition(
                key="color",
                match=models.MatchValue(value="red"),
            ),
        ]
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 None)

-- When using must_not, the clause becomes true if none of the conditions listed inside should is satisfied.
In this sense, must_not is equivalent to the expression (NOT A) AND (NOT B) AND (NOT C).

In [8]:
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must_not=[
            models.FieldCondition(key="city", match=models.MatchValue(value="London")),
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
)

([Record(id=5, payload={'city': 'Moscow', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'Moscow', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)

-- Must and must not combination

In [10]:
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="city", match=models.MatchValue(value="London")),
        ],
        must_not=[
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ],
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)

In [ ]:
##Suppose we have this payload, i.e data which we want to feed in the qdrant vectordatabase
data_to_ingest = [
  { "id": 1, "city": "London", "color": "green" }, #This
  { "id": 2, "city": "London", "color": "red" },
  { "id": 3, "city": "London", "color": "blue" }, #This
  { "id": 4, "city": "Berlin", "color": "red" },
  { "id": 5, "city": "Moscow", "color": "green" },
  { "id": 6, "city": "Moscow", "color": "blue" }
]

##### MatchValue: The simplest kind of condition is one that checks if the stored value equals the given one.

In [11]:
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="city", match=models.MatchValue(value="London")),
        ],
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)

##### MatchAny: The condition will be satisfied if the stored value is matching with atleast one of the given values in the array

In [ ]:
##It will return those records which are having value either city as London or Berlin
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="city", match=models.MatchAny(any=["London","Berlin"])),
        ],
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=2, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 None)

In [14]:
##It will return all records whose city value are not matching with any of the value in given array.
qdrant_client.scroll(
    collection_name="demo_collection1",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="city", match=models.MatchExcept(**{"except":["London","Berlin"]})),
        ],
    ),
)

([Record(id=5, payload={'city': 'Moscow', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'Moscow', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)